## Условие


## Описание модели

Дана выборка $\vec{X} = \{\vec{X}_k\}_{k=1}^K$ сильно зашумленных черно-белых изображений размера $H\times W$ пикселей. Каждое из этих изображений содержит один и тот же неподвижный фон и лицо преступника в неизвестных координатах, при этом лицо попадает в любое изображение целиком. Будем считать, что изоражение лица имеет прямоугольную форму размера $h\times w$ пикселей. Размеры лица на изображении в данных: $w = 75$, $h = 100$.


Введем следующие обозначения:
* $\vec{X}_k(i,j)$ –– пиксель $k$-ого изображения;
* $\vec{B}\in\mathbb{R}^{H\times W}$ –– изображение чистого фона без лица преступника, $\vec{B}(i,j)$ –– пиксель этого изображения;
* $\vec{F}\in\mathbb{R}^{h\times w}$ –– изображение лица преступника, $\vec{F}(i,j)$ –– пиксель этого изображения;
* $\vec{d}_k = (d^h_k,d^w_k)$ –– координаты верхнего левого угла изображения лица на $k$-ом изображении ($d^h_k$ –– по вертикали, $d^w_k$ –– по горизонтали), $\vec{d} = (\vec{d}_1,\dots,\vec{d}_K)$ –– набор координат для всех изображений выборки.  

Также будем считать шум на изображении независимым для каждого пикселя и принадлежащим нормальному распределению $\mathcal{N}(0,s^2)$, где $s$ –– стандартное отклонение. Таким образом для одного изображения имеем:
\begin{equation*}
    p(\vec{X}_k\mid \vec{d}_k, \vec{\theta})=\prod_{ij}
    \begin{cases} 
    	\mathcal{N}(\vec{X}_k(i,j)\mid \vec{F}(i-d_k^h,j-d_k^w),s^2), 
    	& \text{если } (i,j)\in faceArea(\vec{d}_k)\\
    	\mathcal{N}(\vec{X}_k(i,j)\mid \vec{B}(i,j),s^2), & \text{иначе}
    \end{cases},
\end{equation*}
где $\vec{\theta} = \{\vec{B},\vec{F},s^2\}$, $faceArea(\vec{d}_k) = \{(i,j)\mid d_k^h\leq i\leq d_k^h + h-1, d_k^w\leq j\leq d_k^w + w-1\}$.

Распределение на неизвестные координаты лица на изображении зададим общим для всех изображений с помощью матрицы параметров $\vec{A}\in\mathbb{R}^{H-h+1,W-w+1}$ следующим образом:
\begin{equation*}
    p(\vec{d}_k\mid \vec{A})=\vec{A}(d_k^h,d_k^w),\quad\quad\quad   \sum_{ij}A(i,j) = 1,
\end{equation*}
где $\vec{A}(i,j)$ –– элемент матрицы $\vec{A}$.

В итоге имеем следующую совместную вероятностную модель:
\begin{equation*}
    p(\vec{X},\vec{d}\mid \vec{\theta},\vec{A}) = \prod_k 
    p(\vec{X}_k\mid \vec{d}_k,\vec{\theta})p(\vec{d}_k\mid\vec{A}).
\end{equation*}

## Общая формулировка задания

Требуется решить задачу
\begin{equation*}
    p(\vec{X}\mid \vec{\theta},\vec{A})\rightarrow\max_{\vec{\theta},\vec{A}}.
\end{equation*}

Для этого предлагается воспользоваться EM-алгоритмом, то есть перейти к следующей задаче оптимизации нижней оценки на логарифм неполного правдоподобия:
\begin{align*}
    \mathcal L(q,\vec{\theta},\vec{A}) = \mathbb{E}_{q(\vec{d})} \log p(\vec{X},\vec{d}\mid \vec{\theta},\vec{A}) - \mathbb{E}_{q(\vec{d})} \log q(\vec{d}) \rightarrow\max_{q, \vec{\theta},\vec{A}}
\end{align*}
На E-шаге вычисляется оценка на апостериорное распределение на координаты лица на изображениях:
\begin{equation*}
    q(\vec{d}) = p(\vec{d}\mid \vec{X},\vec{\theta},\vec{A}) = \prod_k p(\vec{d}_k\mid \vec{X}_k,\vec{\theta},\vec{A}),
\end{equation*}
а на M-шаге вычисляется точечная оценка на параметры $\vec{\theta},\vec{A}$:
\begin{equation*}
    \mathbb{E}_{q(\vec{d})} \log p(\vec{X},\vec{d}\mid \vec{\theta},\vec{A})
    \rightarrow\max_{\vec{\theta},\vec{A}}.
\end{equation*}

Также далее будет рассматриваться упрощенный вариант ЕМ-алгоритма, который называется hard EM. В нем после Е шага берется не все апостериорное распределение на координаты лица на изображениях, а только МАР оценка на эти координаты (то есть после Е шага $q(\vec{d})$ преобразовывают так, что для каждого изображения $\vec{X}_k$ оценка $q(\vec{d}_k)$ принимает значение 1 только в одной точке –– точке аргмаксимума апостериорного распределения $p(\vec{d}_k\mid \vec{X}_k,\vec{\theta},\vec{A})$). 

При выполнении итераций ЕМ алгоритма нужно следить за значением оптимизируемого функционала $\mathcal L(q,\vec{\theta},\vec{A})$. 


# Ход работы

## Теория

### Формулировка

Вывести формулы для подсчета следующих величин:

1. апостериорного распределения на координаты лица на изображениях $p(\vec{d}_k\mid \vec{X}_k,\vec{\theta},\vec{A})$ на Е-шаге;
2. точечных оценок на параметры $\vec{A}, \vec{\theta}=\{\vec{F},\vec{B},s^2\}$ на М-шаге для ЕМ и МАР-ЕМ алгоритмов (учтите, что точечные оценки здесь нужно получать именно в таком порядке: $\vec{A}, \vec{F},\vec{B},s^2$);
3. нижней оценке на логарифм неполного правдоподобия $\mathcal L(q,\vec{\theta},\vec{A})$.

### Решение

1. апостериорного распределения на координаты лица на изображениях $p(\vec{d}_k\mid \vec{X}_k,\vec{\theta},\vec{A})$ на Е-шаге;


$$
p(\vec{d}_k\mid \vec{X}_k,\vec{\theta},\vec{A}) 
= \frac{p(\vec{X_k}\mid \vec{d}_k \vec{\theta},\vec{A})p(\vec{d}_k \mid \vec{\theta},\vec{A})}{\int{p(\vec{X_k}\mid \vec{d}_k \vec{\theta},\vec{A})p(\vec{d}_k \mid \vec{\theta},\vec{A})d(\vec{d}_k)}}
= \frac{p(\vec{X_k}\mid \vec{d}_k \vec{\theta})p(\vec{d}_k \mid \vec{A})}{\int{p(\vec{X_k}\mid \vec{d}_k \vec{\theta})p(\vec{d}_k \mid \vec{A})d(\vec{d}_k)}}
$$


2. точечных оценок на параметры $\vec{A}, \vec{\theta}=\{\vec{F},\vec{B},s^2\}$ на М-шаге для ЕМ и МАР-ЕМ алгоритмов (учтите, что точечные оценки здесь нужно получать именно в таком порядке: $\vec{A}, \vec{F},\vec{B},s^2$);

\begin{equation*}
\mathbb{E}_{q(\vec{d})} \log p(\vec{X},\vec{d}\mid \vec{\theta},\vec{A}) \rightarrow\max_{\vec{\theta},\vec{A}}
\end{equation*}

\begin{equation*}
\mathbb{E}_{q(\vec{d})} \sum\limits_{k=1}^{K}\log p(\vec{X_k},\vec{d_k}\mid \vec{\theta},\vec{A}) \rightarrow\max_{\vec{\theta},\vec{A}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K}\mathbb{E}_{q(\vec{d})} \log p(\vec{X_k},\vec{d_k}\mid \vec{\theta},\vec{A}) \rightarrow\max_{\vec{\theta},\vec{A}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K}\mathbb{E}_{q_k(\vec{d_k})} \log p(\vec{X_k},\vec{d_k}\mid \vec{\theta},\vec{A}) \rightarrow\max_{\vec{\theta},\vec{A}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K}\mathbb{E}_{q_k(\vec{d_k})} (\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k})+\log p(\vec{d_k}\mid \vec{A}))\rightarrow\max_{\vec{\theta},\vec{A}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K}\int q_k(\vec{d_k}) (\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k})+\log p(\vec{d_k}\mid \vec{A}))d(\vec{d_k}) \rightarrow\max_{\vec{\theta},\vec{A}} 
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) (\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k}=(i, j))+\log p(\vec{d_k}=(i, j)\mid \vec{A}))  \rightarrow\max_{\vec{\theta},\vec{A}} 
\end{equation*}

$$
\text{Так же обозначим оптимальные значения апостериорного распределения}\ p(\vec{d}_k\mid \vec{X}_k,\vec{\theta},\vec{A}))
$$

$$
(i^{opt}_k, j^{opt}_k) = \arg\max(q_k(\vec{d}_k))
$$

* Найдём точечную оценку для $\vec{A}$

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q((i, j))\log p(\vec{d_k}=(i, j)\mid \vec{A})  \rightarrow\max_{\vec{A}} 
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q((i, j))\log A(i, j)\rightarrow\max_{\vec{A}} 
\end{equation*}

\begin{equation*}
\text{воспользуемся результатом с семинара, где мы находили оптимум при помощи лагранжиана}
\end{equation*}

$$
A(i, j) = \frac{\sum\limits_{k=1}^K q_k(i, j)}{K}
$$

$$
A(i, j) = \frac{\sum\limits_{k=1}^K \mathsf{I}(i = i_k^{opt}, j = j_k^{opt})}{K},\ \text{в случае МАР-ЕМ}
$$

* Найдём точечную оценку для $\vec{F}$

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j))\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k}=(i, j))  \rightarrow\max_{\vec{F}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j))\log \prod_{u=0}^{h-1}\prod_{v=0}^{w-1} p(X_k(i+u, j+v)\mid F(u, v), s^2)  \rightarrow\max_{\vec{F}}
\end{equation*}


\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) \sum_{u=0}^{h-1}\sum_{v=0}^{w-1} \log p(X_k(i+u, j+v)\mid F(u, v), s^2)  \rightarrow\max_{\vec{F}}
\end{equation*}

\begin{equation*}
\text{ввиду того, что}\ p(X_k(i+u, j+v)\mid F(u, v), s^2) 
= \frac{1}{s\sqrt{2\pi}}e^{-\frac{\left( X_k(i+u, j+v) - F(u, v)\right)^2}{2s^2}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) \sum_{u=0}^{h-1}\sum_{v=0}^{w-1} (X_k(i+u, j+v)- F(u, v))^2  \rightarrow\min_{\vec{F}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}\sum_{u=0}^{h-1}\sum_{v=0}^{w-1} q_k((i, j)) (X_k(i+u, j+v)- F(u, v))^2  \rightarrow\min_{\vec{F}}
\end{equation*}

\begin{equation*}
\text{приравняем частную производную по }\ F(x, y)\ \text{к нулю}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) (X_k(i+x, j+y)- F(x, y)) = 0
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) X_k(i+x, j+y)
= \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) F(x, y)
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) X_k(i+x, j+y)
= K \cdot F(x, y)
\end{equation*}

\begin{equation*}
F(x, y) = \frac{\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) X_k(i+x, j+y)}{K}
\end{equation*}

\begin{equation*}
F(x, y) = \frac{\sum\limits_{k=1}^{K} X_k(i_k^{opt}+x, j_k^{opt}+y)}{K},\ \text{в случае МАР-ЕМ}
\end{equation*}

* Найдём точечную оценку для $\vec{B}$

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=-0}^{W-w} q_k((i, j))\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k}=(i, j))  \rightarrow\max_{\vec{B}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j))\log \prod_{(u, v) \not\in faceArea((i, j))}p(X_k(u, v)\mid B(u, v), s^2)  \rightarrow\max_{\vec{B}}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}  \sum_{(u, v) \not\in faceArea((i, j))} q_k((i, j)) \log p(X_k(u, v)\mid B(u, v), s^2)  \rightarrow\max_{\vec{B}}
\end{equation*}

\begin{equation*}
\text{аналогично пункту про F}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}  \sum_{(u, v) \not\in faceArea((i, j))} q_k((i, j)) \cdot(X_k(u, v) - B(u, v))^2  \rightarrow\min_{\vec{B}}
\end{equation*}

\begin{equation*}
\text{приравняем частную производную по }\ B(x, y)\ \text{к нулю}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}  \mathsf{I}\left[(x, y) \not\in faceArea((i, j))\right] q_k((i, j)) \cdot(X_k(x, y) - B(x, y)) = 0
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}  \mathsf{I}\left[(x, y) \not\in faceArea((i, j))\right] q_k((i, j)) \cdot(X_k(x, y)
= \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}  \mathsf{I}\left[(x, y) \not\in faceArea((i, j))\right] q_k((i, j))B(x, y)
\end{equation*}

$$
B(x, y) = 
\frac{
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=0}^{H-h}
            \sum\limits_{j=0}^{W-w}
                \mathsf{I}
                \left[
                    (x, y)
                    \not\in
                    faceArea((i, j))
                \right]
                q_k((i, j))
                \cdot X_k(x, y)
}{
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=0}^{H-h}
            \sum\limits_{j=0}^{W-w}
                \mathsf{I}
                    \left[
                        (x, y)
                        \not\in
                        faceArea((i, j))
                    \right]
                    q_k((i, j))
}
$$

$$
B(x, y) = 
\frac{
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=0}^{H-h}
            \sum\limits_{j=0}^{W-w}
                q_k((i, j))
                \cdot X_k(x, y)
    -
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=0}^{H-h}
            \sum\limits_{j=0}^{W-w}
                \mathsf{I}
                \left[
                    (x, y)
                    \in
                    faceArea((i, j))
                \right]
                q_k((i, j))
                \cdot X_k(x, y)
}{
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=0}^{H-h}
            \sum\limits_{j=0}^{W-w}
                q_k((i, j))
    -
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=0}^{H-h}
            \sum\limits_{j=0}^{W-w}
                \mathsf{I}
                \left[
                    (x, y)
                    \in
                    faceArea((i, j))
                \right]
                q_k((i, j))
}
$$


$$
B(x, y) = 
\frac{
    \sum\limits_{k=1}^{K}
        \cdot X_k(x, y)
    -
    \sum\limits_{k=1}^{K}
        X_k(x, y)
        \cdot
        \left(
            \sum\limits_{i=\max(x-h+1, 0}^{x}
                \sum\limits_{j=\max(y-w+1, 0)}^{y}
                    q_k((i, j))
        \right)
}{
    K
    -
    \sum\limits_{k=1}^{K}
        \sum\limits_{i=\max(x-h+1, 0}^{x}
            \sum\limits_{j=\max(y-w+1, 0)}^{y}
                q_k((i, j))
}
$$

$$
B(x, y) = 
\frac{
    \sum\limits_{k=1}^{K}
        \cdot X_k(x, y)
    -
    \sum\limits_{k=1}^{K}
        X_k(x, y)
        \cdot
        \mathsf{I}
        \left[
            (x, y) \in faceArea((i_k^{opt}, j_k^{opt}))
        \right]
}{
    K
    -
    \sum\limits_{k=1}^{K}
        X_k(x, y)
        \cdot
        \mathsf{I}
        \left[
            (x, y) \in faceArea((i_k^{opt}, j_k^{opt}))
        \right]
},\ \text{в случае МАР-ЕМ}
$$


* Найдём точечную оценку для $s^2$

\begin{equation*}
\text{сперва обозначим совмещённое изображение }\ \mu_{i, j}(u, v) =
    \begin{cases} 
    	\vec{F}(u - i,v - j),
    	& \text{если } (u,v)\in faceArea((i, j))\\
    	\vec{B}(u,v), & \text{иначе}
    \end{cases},
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j))\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k}=(i, j))  \rightarrow\max_{s^2}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j))\log p(\vec{X_k}\mid ,\vec{\mu}_{i, j}, s^2)  \rightarrow\max_{s^2}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j))\sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}\log p(X_k(u, v)\mid \mu_{i, j}(u, v), s^2)  \rightarrow\max_{s^2}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j))\log p(X_k(u, v)\mid \mu_{i, j}(u, v), s^2)  \rightarrow\max_{s^2}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j))\log \mathcal{N}(X_k(u, v)\mid \mu_{i, j}(u, v), s^2)  \rightarrow\max_{s^2}
\end{equation*}

$$
\text{ввиду того, что}\ \log \mathcal{N}(X_k(u, v)\mid \mu_{i, j}(u, v), s^2)
= -\left(\frac{\left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2}{2s^2} + \log s + \frac{1}{2}\log{2\pi}\right)
$$

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j)) \left(\frac{\left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2}{2s^2} + \log s + \frac{1}{2}\log{2\pi}\right) \rightarrow\min_{s^2}
\end{equation*}

\begin{equation*}
\text{приравняем производную по }\ s\ \text{к нулю}
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j)) \left(-\frac{\left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2}{s^3} + \frac{1}{s}\right) = 0
\end{equation*}

\begin{equation*}
\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j))s^2 = \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j)) \left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2
\end{equation*}

\begin{equation*}
{KWHs}^2 = \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j)) \left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2
\end{equation*}

\begin{equation*}
s^2 = \frac{\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j)) \left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2}{KWH}
\end{equation*}


\begin{equation*}
s^2 = \frac{\sum\limits_{k=1}^{K} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1} \left(X_k(u, v) - \mu_{i_k^{opt}, j_k^{opt}}(u, v)\right)^2}{KWH},\ \text{в случае МАР-ЕМ}
\end{equation*}

3. нижней оценке на логарифм неполного правдоподобия $\mathcal L(q,\vec{\theta},\vec{A})$.



$$
\mathcal L(q,\vec{\theta},\vec{A}) = \mathbb{E}_{q(\vec{d})} \log p(\vec{X},\vec{d}\mid \vec{\theta},\vec{A}) - \mathbb{E}_{q(\vec{d})} \log q(\vec{d})
$$

$$
\text{распишем каждое слагаемое}
$$

$$
\mathbb{E}_{q(\vec{d})} \log p(\vec{X},\vec{d}\mid \vec{\theta},\vec{A}) 
\\ \stackrel{\text{см пункт 2}}{=} \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) (\log p(\vec{X_k}\mid \vec{\theta},\vec{d_k}=(i, j))+\log p(\vec{d_k}=(i, j)\mid \vec{A}))
\\= \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}q_k((i, j))(\log p(\vec{X_k}\mid \vec{\mu}_{i, j}, s^2)+\log A(i, j))
\\= \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}q_k((i, j))\left( \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}\log \mathcal{N}(\vec{X_k}(u, v)\mid \vec{\mu}_{i, j}(u, v), s^2)+\log A(i, j)\right)
\\=\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}q_k((i, j))\log A(i, j)
    - \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}\sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j))\left(\frac{\left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2}{2s^2} + \frac{1}{2}\log{2\pi s^2}\right)
\\=\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}q_k((i, j))\log A(i, j)
    - \frac{1}{2s^2}\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}\sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j))\left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2 - KWH\frac{1}{2}\log{2\pi s^2}
$$

$$
\mathbb{E}_{q(\vec{d})} \log q(\vec{d}) 
= \sum\limits_{k=1}^{K} \mathbb{E}_{q(\vec{d_k})} \log q(\vec{d_k}) =  \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w} q_k((i, j)) \log q_k((i, j))
$$


$$
\text{получим итоговое выражение}
$$

$$
\mathcal L(q,\vec{\theta},\vec{A}) = \sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}q_k((i, j))(\log A(i, j) - \log q_k((i, j)))
    - \frac{1}{2s^2}\sum\limits_{k=1}^{K} \sum\limits_{i=0}^{H-h} \sum\limits_{j=0}^{W-w}\sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}q_k((i, j))\left(X_k(u, v) - \mu_{i, j}(u, v)\right)^2 - KWH\frac{1}{2}\log{2\pi s^2}
$$

$$
\mathcal L(q,\vec{\theta},\vec{A}) 
= \sum\limits_{k=1}^{K} \log A(i_k^{opt}, j_k^{opt})
    - \frac{1}{2s^2}\sum\limits_{k=1}^{K} \sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}\left(X_k(u, v) - \mu_{i_k^{opt}, j_k^{opt}}(u, v)\right)^2 - KWH\frac{1}{2}\log{2\pi s^2}
    ,\ \text{в случае МАР-ЕМ}
$$

4. Для удобства посчитаем ещё логарифм правдоподобия $\log p(\vec{X} \mid F, B, s^2)$ для $(dh, dw) = (i, j)$:




$$
\sum\limits_{u = 0}^{H-1}\sum\limits_{v = 0}^{W-1}\log \mathcal{N}(\vec{X}(u, v)\mid \vec{\mu}_{i, j}(u, v), s^2)
$$

$$
-
\sum\limits_{u = 0}^{H-1} {
    \sum\limits_{v = 0}^{W-1} {
        \left(
            \frac{
               \left(
                   X(u, v) - \mu_{i, j}(u, v)
               \right)^2
            }
            {
                2s^2
            }
            +
            \frac{
                1
            }{
                2
            }
            \log{2\pi s^2}
        \right)
    }
}
$$

$$
\frac{
    -1
}{
    2s^2
}
\sum\limits_{u = 0}^{H-1} {
    \sum\limits_{v = 0}^{W-1} {
       \left(
           X(u, v) - \mu_{i, j}(u, v)
       \right)^2
    }
}
-
\frac{
    WH
}{
    2
}
\log{2\pi s^2}
$$

* **Примечание:** в работе несколько раз встретилась одна и та же матрица, значения в позиции $(i, j)$ равны:
$$
\sum\limits_{u = 0}^{H-1} {
    \sum\limits_{v = 0}^{W-1} {
        \left(
            X(u, v) - \mu_{i, j}(u, v)
        \right)
        ^2
    }
}
$$

Научимся считать её с помощью свёртки. Для этого представим опять $\mu_{i, j}(u, v)$ через $B$ и $F$

$$
\sum\limits_{(u, v)\in faceArea((i, j))} {
    \left(
        X(u, v)
        -
        F(u-i, v-j)
    \right)
    ^2
}
+
\sum\limits_{(u, v)\not\in faceArea((i, j))} {
    \left(
        X(u, v) - B(u, v)
    \right)
    ^2
}
$$

Для этого научимся считать слагаемые по-очереди. Сначала первое

$$
\sum\limits_{x=0}^{h-1} {
    \sum\limits_{y=0}^{w-1} {
        \left(
            X(i+x, j+y)^2
            -
            2X(i+x, j+y)F(x, y)
            +
            F(x, y)^2
        \right)
    }
}
$$

$$
Convolution(Prod(X, X), \mathbb{1}_{h \times w})(i, j)
-2
Convolution(X, F)(i, j)
+
\sum\limits_{x, y} {
    F(x, y)^2
}
$$


А теперь посчитаем второе слагаемое, прибавивляя и вычитая сумму по $faceArea$:
$$
\sum\limits_{u, v} {
    \left(
        X(u, v) - B(u, v)
    \right)
    ^2
}
-
\sum\limits_{(u, v)\in faceArea((i, j))} {
    \left(
        X(u, v) - B(u, v)
    \right)
    ^2
}
$$

$$
\sum\limits_{u, v} {
    \left(
        X(u, v) - B(u, v)
    \right)
    ^2
}
-
Convolution(Prod((X - B), (X - B)), \mathbb{1}_{h \times w})(i, j)
$$

## Практика

### 1 вопрос

Протестируйте полученный EM алгоритм на сгенерированных данных. Сильно ли влияет начальное приближение на параметры на результаты работы? Стоит ли для данной задачи запускать ЕМ алгоритм из разных начальных приближений?

### 2 вопрос

Запустите ЕМ алгоритм на сгенерированных выборках разных размеров и с разным уровнем зашумления. Как изменения в обучающей выборке влияют на результаты работы (получаемые $F,B$ и $\mathcal L(q,\vec{\theta},\vec{A})$)? При каком уровне шума ЕМ-алгоритм перестает выдавать вменяемые результаты? В данном пункте учтите, что для сравнения значения $\mathcal L(q,\vec{\theta},\vec{A})$ для выборок разного размера стоит нормировать его на объем выборки. 

### 3 вопрос
Сравните качество и время работы ЕМ и hard ЕМ на сгенерированных данных. Как Вы думаете, почему разница в результатах работы так заметна?

### 4 вопрос
Примените ЕМ алгоритм к данным с зашумленными снимками преступника. Приведите результаты работы алгоритма на выборках разного размера. 

### 5 вопрос
Предложите какую-нибудь модификацию полученного ЕМ алгоритма, которая бы работала на данной задаче качественнее и/или быстрее.

**Ответ:** Мне кажется, что можно улучшить распределение позиции лица. Вместо матрицы $A$ использовать, например, какое-нибудь гауссовское распределение. Благодаря такому подходу можно для каждой картинки обучать свои параметры распределения, так как их будет линейное число от количества картинок